In [6]:
from hope.dataset.dataset import CMPFacadeDataset
from hope.models.hope import HopeModel

In [8]:
model = HopeModel(512)

In [2]:
dataset = CMPFacadeDataset('/home/penguin/SSIW/data/base')

In [9]:
for e in dataset:
    image, mask = e
    break

In [10]:
logit, embed = model(image)

In [11]:
embed.shape

torch.Size([1, 512, 736, 960])

In [4]:
image.shape

(448, 583, 3)

In [5]:
mask.shape

torch.Size([14, 448, 583])

In [ ]:
from hope.models.segmodel import SegModel
import torch
from hope.utils.embed import create_embs_from_names
import cv2
from hope.utils.transform_utils import resize_by_scaled_short_side
import json
from hope.utils.transform_utils import single_scale_single_crop_cuda


In [ ]:
model = SegModel(criterions=None,
                     num_classes=512,
                     load_imagenet_model=False, imagenet_ckpt_fpath='')


model = model.eval()

model = torch.nn.DataParallel(model)


In [ ]:
checkpoint = torch.load('/home/penguin/SSIW/data/model/segformer_7data.pth', map_location='cpu')['state_dict']
ckpt_filter = {k: v for k, v in checkpoint.items() if 'criterion.0.criterion.weight' not in k}
maybe_errors = model.load_state_dict(ckpt_filter, strict=False)

In [ ]:
model = model.to('cuda')

In [ ]:
with open('/home/penguin/SSIW/Test_Minist/cmp_dataset_desc.json', 'r', encoding='utf-8') as f:
    new_definitions = json.load(f)

In [ ]:
user_label = ['background','facade','window','door', 'cornice', 'sill', 'balcony', 'blind','deco', 'molding', 'pillar', 'shop', 'unlabel']

In [ ]:
gt_embs_list = create_embs_from_names(user_label, new_definitions).float()

In [ ]:
rgb = cv2.imread('/home/penguin/SSIW/data/base/cmp_b0001.jpg', -1)[:, :, ::-1]

In [ ]:
image_resized = resize_by_scaled_short_side(rgb, 720, 1)
h, w, _ = rgb.shape

In [ ]:
out_logit, emb = single_scale_single_crop_cuda(model, image_resized, h, w, gt_embs_list=gt_embs_list, args=None)

In [ ]:
import numpy as np

In [ ]:
id_to_label = {i: v for i, v in enumerate(user_label)}
id_to_label[255] = 'unlabel'

In [ ]:
prediction.shape

In [ ]:
prediction

In [ ]:
prediction = out_logit.argmax(axis=-1).squeeze()   # (h, w)
probs = out_logit.max(axis=-1).squeeze()  # (h, w)
high_prob_mask = probs > 0.5

mask = high_prob_mask
prediction[~mask] = 255

labels = np.unique(prediction)
label_names = {i: id_to_label[i] for i in labels}
# label_names = [id_to_label[i] for i in labels]
print()
print(f'label_names for img', label_names)

In [ ]:
id_to_label

In [ ]:
prediction.shape

In [ ]:
import torch.nn as nn
import torch

In [ ]:
loss = nn.CrossEntropyLoss()

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

In [ ]:
target

In [ ]:
input

In [1]:
from hope.models.hope import HopeModel
from hope.dataset.dataset import CMPFacadeDataset

/home/penguin/miniconda3/envs/research/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
model = HopeModel()

In [3]:
dataset = CMPFacadeDataset('/home/penguin/SSIW/data/base')

In [4]:
out_logits, embed =model(dataset[0][0])

In [ ]:
def generate_label(image_path, metadata):
    path = r'/home/penguin/SSIW/data/base/cmp_b0001.jpg'
    image = cv2.imread(path)

In [9]:
import cv2

In [28]:
num_class = 12
path = r'/home/penguin/SSIW/data/base/cmp_b0346.jpg'
image = cv2.imread(path)
height, width, _ = image.shape

In [77]:
import torch

In [78]:
mask = torch.zeros((num_class + 1, height, width))

In [79]:
mask[0].fill_(1)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [80]:
for obj_meta in dataset[0][1]:
    x1 = round(obj_meta.points.x1 * height)
    x2 = round(obj_meta.points.x2 * height)
    y1 = round(obj_meta.points.y1 * width)
    y2 = round(obj_meta.points.y2 * width)
    mask[obj_meta.label][x1:x2+1, y1:y2+1] = 1
    mask[0][x1:x2+1, y1:y2+1] = 0

In [84]:
mask.shape

torch.Size([13, 448, 583])

In [7]:
dataset[0][1]

[ObjectMetadata(points=Point(x1=0.0066964, x2=0.99777, y1=0.06175, y2=0.9777), label=2, label_name='facade', flag=1),
 ObjectMetadata(points=Point(x1=0.13393, x2=0.19643, y1=0.2813, y2=0.32075), label=3, label_name='window', flag=1),
 ObjectMetadata(points=Point(x1=0.15848, x2=0.18973, y1=0.14065, y2=0.21441), label=3, label_name='window', flag=1),
 ObjectMetadata(points=Point(x1=0.13393, x2=0.19643, y1=0.71698, y2=0.75643), label=3, label_name='window', flag=1),
 ObjectMetadata(points=Point(x1=0.15848, x2=0.18973, y1=0.81647, y2=0.88336), label=3, label_name='window', flag=1),
 ObjectMetadata(points=Point(x1=0.30804, x2=0.42188, y1=0.16295, y2=0.22298), label=3, label_name='window', flag=1),
 ObjectMetadata(points=Point(x1=0.3058, x2=0.41964, y1=0.36192, y2=0.42024), label=3, label_name='window', flag=1),
 ObjectMetadata(points=Point(x1=0.51339, x2=0.6317, y1=0.16295, y2=0.22127), label=3, label_name='window', flag=1),
 ObjectMetadata(points=Point(x1=0.51339, x2=0.6317, y1=0.36192, y2

In [6]:
embed

tensor([[0.0080, 0.0077, 0.0075,  ..., 0.0030, 0.0032, 0.0034],
        [0.0084, 0.0081, 0.0078,  ..., 0.0030, 0.0033, 0.0035],
        [0.0088, 0.0085, 0.0082,  ..., 0.0031, 0.0034, 0.0037],
        ...,
        [0.0361, 0.0354, 0.0348,  ..., 0.0353, 0.0349, 0.0344],
        [0.0362, 0.0357, 0.0351,  ..., 0.0353, 0.0348, 0.0342],
        [0.0364, 0.0359, 0.0354,  ..., 0.0353, 0.0346, 0.0339]],
       device='cuda:0')

In [66]:
import torch

# create a random tensor with shape (n, h, w)
n, h, w = 5, 10, 10
tensor = torch.zeros((n, h, w))

# set some random elements to 1 in other dimensions
tensor[1, 3, 4] = 1
tensor[2, 8, 2] = 1
tensor[4, 1, 9] = 1

# check if all other elements in the first dimension except for the first one are 0
if torch.sum(tensor[1:] != 0) == 0:
    # fill (0, h, w) with 1s
    tensor[0, h-1, w-1] = 1